In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.v2 as transforms
import torchvision.io as tv_io

import glob
from PIL import Image

import utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

## 7.1 Load Model

In [19]:
from torchvision.models import vgg16
from torchvision.models import VGG16_Weights

weights = VGG16_Weights.DEFAULT
vgg_model = vgg16(weights=weights)

In [20]:
# Freeze base model
vgg_model.requires_grad_(False)
next(iter(vgg_model.parameters())).requires_grad

False

## 7.2 Add Layers to Model

In [73]:
vgg_model.classifier[0:3]

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
)

In [40]:
N_CLASSES = 6

my_model = nn.Sequential(
    vgg_model.features,
    vgg_model.avgpool,
    nn.Flatten(),
    vgg_model.classifier[0:3],
    nn.Linear(4096, 500),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(500,500),
    nn.ReLU(),
    nn.Linear(500, N_CLASSES)
)
my_model

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## 7.3 Compile Model

In [41]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(my_model.parameters())
my_model = torch.compile(my_model.to(device))

## 7.4 Data Transforms

In [42]:
pre_trans = weights.transforms()

In [43]:
IMG_WIDTH, IMG_HEIGHT = (224, 224)

random_trans = transforms.Compose([
    transforms.RandomRotation(18),
    transforms.RandomResizedCrop((IMG_WIDTH, IMG_HEIGHT), scale=(0.9,1), ratio=(1,1)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=.1, contrast=.1, saturation=.1, hue=.1)
])

## 7.5 Load Dataset

In [44]:
DATA_LABELS = ["freshapples", "freshbanana", "freshoranges", "rottenapples", "rottenbanana", "rottenoranges"] 
    
class MyDataset(Dataset):
    def __init__(self, data_dir):
        self.imgs = []
        self.labels = []
        
        for l_idx, label in enumerate(DATA_LABELS):
            data_paths = glob.glob(data_dir + label + '/*.png', recursive=True)
            for path in data_paths:
                img = tv_io.read_image(path, tv_io.ImageReadMode.RGB)
                self.imgs.append(pre_trans(img).to(device))
                self.labels.append(torch.tensor(l_idx).to(device))


    def __getitem__(self, idx):
        img = self.imgs[idx]
        label = self.labels[idx]
        return img, label

    def __len__(self):
        return len(self.imgs)

In [45]:
n = 32

train_path = "data/fruits/train/"
train_data = MyDataset(train_path)
train_loader = DataLoader(train_data, batch_size=n, shuffle=True)
train_N = len(train_loader.dataset)

valid_path = "data/fruits/valid/"
valid_data = MyDataset(valid_path)
valid_loader = DataLoader(valid_data, batch_size=n, shuffle=False)
valid_N = len(valid_loader.dataset)

## 7.6 Train the Model

In [46]:
epochs = 20

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    utils.train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
    utils.validate(my_model, valid_loader, valid_N, loss_function)

Epoch: 0
Train - Loss: 30.8188 Accuracy: 0.6709
Valid - Loss: 8.0815 Accuracy: 0.7933
Epoch: 1
Train - Loss: 16.1231 Accuracy: 0.8477
Valid - Loss: 9.5559 Accuracy: 0.7872
Epoch: 2
Train - Loss: 14.3205 Accuracy: 0.8545
Valid - Loss: 6.5299 Accuracy: 0.8602
Epoch: 3
Train - Loss: 12.8707 Accuracy: 0.8739
Valid - Loss: 3.3696 Accuracy: 0.9210
Epoch: 4
Train - Loss: 10.5644 Accuracy: 0.8968
Valid - Loss: 4.4990 Accuracy: 0.9088
Epoch: 5
Train - Loss: 11.6864 Accuracy: 0.8883
Valid - Loss: 4.0577 Accuracy: 0.9240
Epoch: 6
Train - Loss: 10.8877 Accuracy: 0.8917
Valid - Loss: 2.5819 Accuracy: 0.9392
Epoch: 7
Train - Loss: 9.9469 Accuracy: 0.8959
Valid - Loss: 5.0297 Accuracy: 0.8815
Epoch: 8
Train - Loss: 12.7429 Accuracy: 0.8680
Valid - Loss: 7.2819 Accuracy: 0.8693
Epoch: 9
Train - Loss: 9.6624 Accuracy: 0.9010
Valid - Loss: 4.6512 Accuracy: 0.8967
Epoch: 10
Train - Loss: 8.8804 Accuracy: 0.9179
Valid - Loss: 3.4135 Accuracy: 0.9240
Epoch: 11
Train - Loss: 9.7378 Accuracy: 0.9019
Valid - 

## 7.7 Unfreeze Model for Fine Tuning

In [47]:
# Unfreeze the base model
vgg_model.requires_grad_(True)
optimizer = Adam(my_model.parameters(), lr=.0001)

In [39]:
# Unfreeze the base model
vgg_model.requires_grad_(False)
optimizer = Adam(my_model.parameters())
next(iter(vgg_model.parameters())).requires_grad

False

In [48]:
epochs = 1

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    utils.train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
    utils.validate(my_model, valid_loader, valid_N, loss_function)

Epoch: 0
Train - Loss: 6.4352 Accuracy: 0.9416
Valid - Loss: 3.8839 Accuracy: 0.9210


## 7.8 Evaluate the Model

In [ ]:
utils.validate(my_model, valid_loader, valid_N, loss_function)